This file will convert our dummy inputs into dataframes.

In [11]:
# Imports
import json
import pandas as pd
import uuid

General setup functions

In [12]:
# Generate UUID
def generate_uuid(first_name, last_name):
    """Generate a deterministic UUID based on athlete attributes."""
    namespace = uuid.UUID("123e4567-e89b-12d3-a456-426614174000")  # Fixed namespace UUID
    unique_string = f"{first_name.lower()}_{last_name.lower()}"
    return str(uuid.uuid5(namespace, unique_string))

# Loading in imports
def load_json(file_path):
    with open(file_path, "r") as file:
        return json.load(file)

def load_csv(csv_path):
    return pd.read_csv(csv_path)

# Formatting exports
def export_to_json(dataframes, output_path):
    formatted_output = {key: df.to_dict(orient="records") for key, df in dataframes.items()}
    with open(output_path, "w") as outfile:
        json.dump(formatted_output, outfile, indent=4)
    print(f"Data exported to {output_path}")

def coalesce_rows(group: pd.DataFrame) -> pd.Series:
    # For each column in the group, pick the first non-empty value.
    # (Non-empty meaning not NaN and not an empty string.)
    merged = {}
    for col in group.columns:
        non_nulls = group[col].dropna().loc[group[col] != ""]
        if len(non_nulls) > 0:
            merged[col] = non_nulls.iloc[0]
        else:
            merged[col] = ""
    return pd.Series(merged)

def coalesce_df(df: pd.DataFrame, key_col: str) -> pd.DataFrame:
    # Given a dataframe and a column (key_col) that identifies the entity,
    # group by that key and merge partial rows into a single row per key.
    if key_col not in df.columns or df.empty:
        return df
    
    df_coalesced = (
        df.groupby(key_col, as_index=False)
          .apply(coalesce_rows) #, include_groups=False)
          .reset_index(drop=True)
    )
    return df_coalesced

Conversion functions

In [13]:
def json_to_dataframe(data):

    df_csv = load_csv("inputs/dummy_ids.csv").fillna("")

    csv_lookup = {}
    for _, row in df_csv.iterrows():
        # Make a key from the CSV row's first & last name in lowercase
        key = (str(row["First Name"]).strip().lower(), str(row["Last Name"]).strip().lower())
        csv_lookup[key] = {
            "is_current": row.get("is_current", ""),
            "Catapult ID": row.get("Catapult ID", ""),
            "VALD ID": row.get("VALD ID", ""),
            "UFID": row.get("UFID", "")
        }

    athlete_uuid_map = {}  # Dictionary to track unique athletes
    all_athletes = []
    
    # Catapult
    for athlete in data.get("athletes", []):
        first_name = athlete.get("first_name", "").strip()
        last_name = athlete.get("last_name", "").strip()
        athlete_key = (first_name.lower(), last_name.lower())
        if athlete_key not in athlete_uuid_map:
            athlete_uuid_map[athlete_key] = generate_uuid(first_name, last_name)

        csv_info = csv_lookup.get(athlete_key, {})    
        
        date_of_birth = athlete.get("date_of_birth_date", "").strip()
        year_of_birth = date_of_birth[:4] if date_of_birth else athlete.get("year_of_birth_date", "").strip()
        
        all_athletes.append({
            "Athlete UUID": athlete_uuid_map[athlete_key],
            "First Name": first_name,
            "Last Name": last_name,
            "Sex": athlete.get("gender", ""),
            "Home State": athlete.get("home_state", ""),
            "Home Town": athlete.get("home_town", ""),
            "High School": athlete.get("highschool", ""),
            "Date of Birth": date_of_birth,
            "Year of Birth": year_of_birth,
            "Is Current": csv_info.get("is_current", "")
        })
    
    # ForceDecks
    for athlete in data.get("forcedecks_athletes", []):
        first_name = athlete.get("givenName", "").strip()
        last_name = athlete.get("familyName", "").strip()
        athlete_key = (first_name.lower(), last_name.lower())
        if athlete_key not in athlete_uuid_map:
            athlete_uuid_map[athlete_key] = generate_uuid(first_name, last_name)

        csv_info = csv_lookup.get(athlete_key, {})    
        
        all_athletes.append({
            "Athlete UUID": athlete_uuid_map[athlete_key],
            "First Name": first_name,
            "Last Name": last_name,
            "Sex": athlete.get("gender", ""),
            "Home State": athlete.get("home_state", ""),
            "Home Town": "",
            "High School": "",
            "Date of Birth": "",
            "Year of Birth": "",
            "Is Current": csv_info.get("is_current", "")
        })
    
    # Build raw df of partial athlete info
    df_athletes_raw = pd.DataFrame(all_athletes)

    dataframes = {}
    
    dataframes["Athletes"] = df_athletes_raw

    ids_rows = []
    for ath in all_athletes:
        key = (ath["First Name"].lower(), ath["Last Name"].lower())
        csv_info = csv_lookup.get(key, {})
        ids_rows.append({
            "Athlete UUID": ath["Athlete UUID"],
            "Catapult ID": csv_info.get("Catapult ID", ""),
            "VALD ID": csv_info.get("VALD ID", ""),
            "UF ID": csv_info.get("UFID", "")
        })
    
    dataframes["IDs"] = pd.DataFrame(ids_rows)

    dataframes["Teams"] = pd.DataFrame(list({
        athlete["teamId"]: {
            "Team ID": athlete["teamId"],
            "Sport": next(
                (attr["valueName"]
                 for attr in athlete.get("attributes", [])
                 if attr["typeName"] == "Sport"
                ), 
                None
            ),
            "Sex": None,
        }
        for athlete in data.get("forcedecks_athletes", [])
        if "teamId" in athlete
    }.values()))

    # Seasons
    dataframes["Seasons"] = pd.DataFrame([
        {
            "Season ID": period["id"],
            "Season year": period["name"],
            "Sport": None
        }
        for period in data.get("periods", [])
    ])

    # Athlete Seasons
    dataframes["Athlete Seasons"] = pd.DataFrame([
        {
            "Athlete Season id": None,
            "Athlete UUID": athlete["id"],
            "Season ID": None,  # Needs mapping
            "Team ID": athlete.get("current_team_id", "")
        }
        for athlete in data.get("athletes", [])
    ])

    # Events
    dataframes["Events"] = pd.DataFrame([
        {
            "Event ID": period["id"],
            "Team ID": None,
            "Event Type": period["name"],
            "Start Unix": period["start_time"],
            "End Unix": period["end_time"],
            "Team Result": None
        }
        for period in data.get("periods", [])
    ])

    # Performances
    dataframes["Performances"] = pd.DataFrame([
        {
            "Performance ID": activity["id"],
            "Event ID": None,  # Needs mapping
            "Athlete UUID": None,  # Needs mapping
            "Athlete opponent": None,
            "Team ID": activity.get("owner_id", ""),
            "Start Unix": activity["start_time"],
            "End Unix": activity["end_time"],
            "Result": None,  # Placeholder for sensor data
        }
        for activity in data.get("activities", [])
    ])

    # Performance Results
    dataframes["Performance Results"] = pd.DataFrame([
        {
            "Performance Result ID": activity["id"],
            "Performance ID": None,  # Needs mapping
            "Modality ID": None,     # Needs mapping
            "Result": None,          # Placeholder for sensor data
            "Is Raw": True
        }
        for activity in data.get("activities", [])
    ])

    # Tests
    dataframes["Tests"] = pd.DataFrame([
        {
            "Test ID": test["id"],
            "Athlete UUID": test["athleteId"],
            "Team ID": None,
            "Modality ID": None,
            "Start Unix": test["startTime"],
            "End Unix": test["endTime"],
            "Date Uploaded": test["recordedUTC"],
            "Test Type": None
        }
        for test in data.get("forcedecks_tests", [])
    ])

    # Test Results
    dataframes["Test Results"] = pd.DataFrame([
        {
            "Test Result ID": f"{test['id']}-{result['resultId']}",
            "Test ID": test["id"],
            "Result": result["value"],
            "Is Raw": True
        }
        for test in data.get("forcedecks_tests", [])
        for result in test["results"]
    ])

    # Modalities
    dataframes["Modalities"] = pd.DataFrame()

    # Athlete Metadata
    dataframes["Athlete Metadata"] = pd.DataFrame([
        {
            "Athlete UUID": athlete["id"],
            "Sex": athlete.get("gender", ""),
            "Jersey": athlete.get("jersey", ""),
            "Velocity Max": athlete.get("velocity_max", None),
            "Acceleration Max": athlete.get("acceleration_max", None),
            "Heart Rate Max": athlete.get("heart_rate_max", None),
            "Player Load Max": athlete.get("player_load_max", None),
            "Weight": athlete.get("weight", None),
            "Height": athlete.get("height", None),
            "Age": athlete.get("age", None),
            "Team ID": athlete.get("current_team_id", ""),
            "Position": athlete.get("position_name", ""),
            "Is Current": True
        }
        for athlete in data.get("athletes", [])
    ])

    # Medical
    dataframes["Medical"] = pd.DataFrame([
        {
            "Medical Record ID": record["id"],
            "Athlete UUID": record["athlete_uuid"],
            "Date": record["date"],
            "Record": record["record"]
        }
        for record in data.get("medical_records", [])
    ])

    # Academic Records
    dataframes["Academic Records"] = pd.DataFrame([
        {
            "Academic Record ID": record["id"],
            "Athlete UUID": record["athlete_uuid"],
            "Date": record["date"],
            "Record": record["record"]
        }
        for record in data.get("academic_records", [])
    ])

    # Consent
    dataframes["Consent"] = pd.DataFrame([
        {
            "Record ID": record["id"],
            "Athlete UUID": record["athlete_uuid"],
            "Consent Status": record["consent_status"]
        }
        for record in data.get("consent", [])
    ])

#This is the list of the dataframes to bring together to make one entry per UUID or Team ID etc
    coalesce_map = {
        "Athletes": "Athlete UUID",
        "Teams": "Team ID",
        "Seasons": "Season ID",
        "Athlete Seasons": "Athlete Season id",
        "Events": "Event ID",
        "Performances": "Performance ID",
        "Performance Results": "Performance Result ID",
        "Tests": "Test ID",
        "Test Results": "Test Result ID",
        "Athlete Metadata": "Athlete UUID",
        "Medical": "Medical Record ID",
        "Academic Records": "Academic Record ID",
        "Consent": "Record ID"
    }

    dataframes["IDs"] = dataframes["IDs"].drop_duplicates()
    
    for df_name, df_obj in dataframes.items():
        key_col = coalesce_map.get(df_name)
        if key_col:  # Only coalesce if we have a mapping
            dataframes[df_name] = coalesce_df(df_obj, key_col)
    
    return dataframes


Implementation

In [14]:
# Load your data
input_json_files = {
    "athletes": "inputs/Catapult/athletes.json",
    "forcedecks_athletes": "inputs/ForceDecks/forcedecks_athletes.json",
    "forcedecks_tests": "inputs/ForceDecks/forcedecks_tests.json",
    "periods": "inputs/Catapult/periods.json",
    "activities": "inputs/Catapult/activities.json"
}

data = {key: load_json(path) for key, path in input_json_files.items()}

# Convert to dataframes and coalesce
dataframes = json_to_dataframe(data)

# Export
output_json_path = "formatted_output.json"
export_to_json(dataframes, output_json_path)

# Check results
for name, df in dataframes.items():
    print(f"--- {name} ---")
    display(df)

Data exported to formatted_output.json
--- Athletes ---


,Athlete UUID,First Name,Last Name,Sex,Home State,Home Town,High School,Date of Birth,Year of Birth,Is Current
0,25c3310f-c13d-5040-a0e7-5f0526a7c96e,Stephanie,Lockwood,,,,,2000-05-18,2000,True
1,324ac544-b506-5bec-95f0-40913f730cd6,Sarah,Gemma,Female,,,,2006-01-01,2006,True
2,ae1d7060-92af-5a83-88d1-270085117855,Celeste,Wilkins,Female,,,,2001-11-09,2001,True
3,b3030aee-4436-5857-96f9-a8e960938a1d,Sarah,Hunt,,,,,,,True
4,c997c090-9374-50e3-bcc7-59079888f075,Stacy,Ardmore,,,,,,,True
5,cc62c471-c03c-5be2-97a0-f65ac7e96f4a,Lorena,Pagani,Unspecified,,,,2002-02-03,2002,True
6,d375eca6-d1a9-5869-bc23-88b4f50be3b2,Ella,Richardson,,,,,,,True
7,d955f40c-5be8-5038-945a-367d87765784,Zacherina,Smith,,,,,,,True
8,db45218b-1527-55c4-96c1-bdda35ffba35,Tabitha,Archibald,Unspecified,,,,2003-07-07,2003,True
9,e0a8c189-6bc4-5207-8334-29b26c48601c,Laurita,Carvallo,Unspecified,,,,2000-10-26,2000,True


--- IDs ---


,Athlete UUID,Catapult ID,VALD ID,UF ID
0,324ac544-b506-5bec-95f0-40913f730cd6,03e10c6e-fce8-44ea-b83d-32d6bbff6339,,31607378
1,ae1d7060-92af-5a83-88d1-270085117855,08ce84d8-54d7-413e-afee-11c3524dc02c,,84677404
2,db45218b-1527-55c4-96c1-bdda35ffba35,0f529ae8-e286-41d7-bb64-1e24de27f857,,96700295
3,e0a8c189-6bc4-5207-8334-29b26c48601c,0f5da3a5-af3e-45bd-a8c1-aabf5fc1b62a,,16320050
4,25c3310f-c13d-5040-a0e7-5f0526a7c96e,12c654e0-6b2e-4d97-9257-7e5128996dc1,,28305466
5,cc62c471-c03c-5be2-97a0-f65ac7e96f4a,13ac9c28-d8f3-4572-b58a-d214af92cfd2,,89469567
6,d375eca6-d1a9-5869-bc23-88b4f50be3b2,,ef069ac4-febb-42ef-8aa2-d6890818a77f,46835886
7,b3030aee-4436-5857-96f9-a8e960938a1d,,77f96b7d-1c16-4d33-ac23-0514d6a4f934,69991321
8,c997c090-9374-50e3-bcc7-59079888f075,,20befb4a-7e37-4ba4-9f72-2107949d8d8a,23475685
9,d955f40c-5be8-5038-945a-367d87765784,,ef08547a-4dd6-4342-9e3f-bb042e7e2e20,83319481


--- Teams ---


...

""


--- Academic Records ---


""


--- Consent ---


""
